In [12]:
from bson.objectid import ObjectId
from pymongo import MongoClient
import numpy as np
import jaro

def test_filter(name, site1, raw_one1, site2, raw_one2):
    if site1 != 'NTIS' and site2 != 'NTIS':
        inst = jaro.jaro_winkler_metric(raw_one1.get('author_inst').split(";")[raw_one1.get('author').split(";").index(name)], raw_one2.get('author_inst').split(";")[raw_one2.get('author').split(";").index(name)])
        weight = 0

        if raw_one1.get('title') == raw_one2.get('title') or inst >= 0.8:
            weight = 4 #6
        else:
            journal1 = raw_one1.get('journal')
            conference1 = raw_one1.get('issue_inst')
            issYear1 = int(raw_one1.get('issue_year')[:4])
            coauthor1 = raw_one1.get('author').split(";")[1:-1]
            keyword1 = raw_one1.get('paper_keyword').replace(" ", "").split(".")

            journal2 = raw_one2.get('journal')
            conference2 = raw_one2.get('issue_inst')
            issYear2 = int(raw_one2.get('issue_year')[:4])
            coauthor2 = raw_one2.get('author').split(";")[1:-1]
            keyword2 = raw_one2.get('paper_keyword').replace(" ", "").split(".")
            
            joc = 1 if journal1 == journal2 and conference1 == conference2 else -1
            yop = -(2*(abs(issYear1-issYear2)/10)-1)
            co_authorship = 1 - np.exp(-len([i for i in coauthor1 if i in coauthor2]))
            keyword = 1 - np.exp(-len([i for i in keyword1 if i in keyword2]))

            print(f'joc: {joc} | yop: {yop} | co_authorship: {co_authorship} | keyword: {keyword}')

            weight += yop + co_authorship + keyword
    else:
        weight = 0
        print('NTIS는 개발중')

    return weight

In [13]:
client = MongoClient('mongodb://203.255.92.141:27017', authSource='admin')

name = '최도진'
site1 = 'Scienceon'
site2 = 'Scienceon'
input_paper1 = ObjectId('61939443e9b04a9d64abe055')
input_paper2 = ObjectId('61939447e9b04a9d64abe06a')

if site1 == 'Scienceon' :
    site_raw1 = client['SCIENCEON']['Rawdata']
elif site1 == 'NTIS' :
    site_raw1 = client['NTIS']['Rawdata']
elif site1 == 'DBPIA' :
    site_raw1 = client['DBPIA']['Rawdata']

if site2 == 'Scienceon' :
    site_raw2 = client['SCIENCEON']['Rawdata']
elif site2 == 'NTIS' :
    site_raw2 = client['NTIS']['Rawdata']
elif site2 == 'DBPIA' :
    site_raw2 = client['DBPIA']['Rawdata']

input_raw1 = site_raw1.find_one({ '_id' : input_paper1 })
input_raw2 = site_raw2.find_one({ '_id' : input_paper2 })

test_filter(name, site1, input_raw1, site2, input_raw2)


4